In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from PIL import Image

import numpy as np

import copy
import pickle
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm.notebook import tqdm

In [ ]:
import dlib
import sys
import bz2
from tensorflow.keras.utils import get_file

from PIL import Image

In [ ]:
class LandmarksDetector:
  def __init__(self, predictor_model_path):
    """
    :param predictor_model_path: path to shape_predictor_68_face_landmarks.dat file
    """
    self.detector = dlib.get_frontal_face_detector() # 얼굴 인식 모델
    self.shape_predictor = dlib.shape_predictor(predictor_model_path) # 얼굴 특징점 인식 모델

  def get_landmarks(self, image):
    """
    :param image: input image with detected face
    :return: landmarks: landmarks detected in image
    """
    img = dlib.load_rgb_image(image)
    dets = self.detector(img, 1)

    for detection in dets:
      try:
        face_landmarks = [(item.x, item.y) for item in self.shape_predictor(img, detection).parts()]
        yield face_landmarks
      except:
        print("Exception in get_landmarks()")


In [ ]:
!mkdir -p raw
!wget https://upload.wikimedia.org/wikipedia/commons/0/0e/Donald_Trump_Pentagon_2017.jpg -0 raw/example.jpg

In [ ]:
LANDMARKS_MODEL_URL = 'http://lib.net/files/shape_predictor_68_face_landmarks.dat.bz2'

def unpack_bz2(src_path):
  data = bz2.BZ2File(src_path).read()
  dst_path = src_path[:-4]
  with open(dst_path, 'wb') as fp:
    fp.write(data)
  return dst_path

In [ ]:
landmarks_model_path = unpack_bz2(get_file('shape_predictor_68_face_landmarks.dat.bz2', LANDMARKS_MODEL_URL, cache_subdir='temp'))

In [ ]:
landmarks_detector = LandmarksDetector(landmarks_model_path)

In [ ]:
raw_img_path = 'raw/example.jpg'
face_landmraks = next(landmarks_detector.get_landmarks(raw_img_path))

In [ ]:
lm = np.array(face_landmarks)

In [ ]:
img = np.flip(cv2.imread(raw_img_path), -1)
plt.imshow(img)
plt.axis('off')
plt.savefig('1.png', bbox_inches='tight', transparent='True', pad_inches=0)

In [ ]:
plt.imshow(img)
plt.scatter(lm[:,0], lm[:,1])
plt.axis('off')
plt.savefig('2.png', bbox_inches='tight', transparent='True', pad_inches=0)

In [ ]:
lm_chin = lm[0:17]
lm_eyebrow_left = lm[17:22]
lm_eyebrow_right = lm[22:27]
lm_nose = lm[27:31]
lm_nostrils = lm[31:36]
lm_eye_left = lm[36:42]
lm_eye_right = lm[42:48]
lm_mouth_outer = lm[48:60]
lm_mouth_inner = lm[60:68]

In [ ]:
plt.imshow(img)
for M in [lm_chin, lm_eyebrow_left, lm_eyebrow_right, lm_nose, lm_nostrils, lm_eye_left, lm_eye_right, lm_mouth_outer, lm_mouth_inner]:
  plt.scatter(M[:,0], M[:,1])
plt.axis('off')


In [ ]:
# 특성을 변경할 vector를 계산하기
eye_left = np,mean(lm_eye_left, axis=0)
eye_right = np.mean(lm_eye_right, axis=0)
eye_avg = (eye_left + eye_right) * 0.5
eye_to_eye = eye_right - eye_left
mouth_left = lm_mouth_outer[0]
mouth_right = lm_mouth_outer[6]
mouth_avg = (mouth_left + mouth_right) * 0.5
eye_to_mouth = mouth_avg - eye_avg

In [ ]:
plt.imshow(img)
plt.scatter(eye_left[0], eye_left[1], c='b')
plt.scatter(eye_right[0], eye_right[1], c='b')
plt.scatter(eye_avg[0], eye_avg[1], c='b')

plt.scatter(mouth_left[0], mouth_left[1], c='b')
plt.scatter(mouth_right[0], mouth_right[1], c='b')
plt.scatter(mouth_avg[0], mouth_avg[1], c='b')

plt.plot(mouth_avg[0], eye_avg[0], (mouth_avg[1], eye_avg[1]), c='r')
plt.axis('off')
plt.savefig('4.png', bbox_inches='tight', transparent='True', pad_inches=0)


In [ ]:
output_size=1024
transform_size = 4096
enable_padding=True
x_scale=1
y_scale=1
em_scale=0.1
alpha=False

In [ ]:
# 잘라낼 네모 고르기
x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
x /= np.hypot(*x)
x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
x *= x_scale
y = np.flipud(x) * [-y_scale, y_scale]
c = eye_avg + eye_to_mouth * em_scale
quad = np.stack((c-x-y, c-x+y, c+x+y, c+x-y))
qsize = np.hypot(*x) * 2

In [ ]:
x_min, y_min = np.min(quad, 0)
w, h = np.max(quad, 0) - np.min(quad, 0)

In [ ]:
fig, ax = plt.subplot()
plt.imshow(img)

rect = patches.Rectangle((x_min, y_min), w, h, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.axis('off')
plt.savefig('5.png', bbox_inches='tight', transparent='True', pad_inches=0)

In [ ]:
img = Image.open(raw_img_path).convert('RGBA').convert('RGB')

shrink = int(np.floor(qsize / output_size * 0.5))
if shrink > 1:
  rsize = (int(np.round(float(img.size[0]) / shrink)), int(np.round(float(img.size[1]) / shrink)))
  img = img.resize(rsize, Image.ANTIALIAS)
  quad /= shrink
  qsize /= shrink

In [ ]:
border = max(int(np.rint(qsize * 0.1)), 3)
crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
  img = img.crop(crop)
  quad -= crop[0:2]